
# Prueba de difusores

Stable Diffusion

Link:  https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5

Stable Diffusion es un modelo de aprendizaje automático desarrollado por Runway y LMU Múnich​ para generar imágenes digitales de alta calidad a partir de descripciones en lenguaje natural o estímulos.

Para poder usar las ventajas de Stable Difussion se necesita un procesador que permita realizar operaciones especiales con el modelo, es necesario un acelerador de procesamiento gráfico, como el de las tarejetas de aceleración gráfica para los videojuegos, se debe elegir T4 GPU en el entorno de ejecución.

# Herramientas de apoyo

**Transformers** es una librería que facilita el uso de los modelos largos de lenguajes, sentencepiece es otra herramientaque que permite codificar y decodificar en tokens las consultas que se le hacen a un modelo largo de lenguaje, y accelerate permite detectar automáticamente el tipo de configuración distribuida que se tiene disponible en una máquina virtual e inicializará todos los componentes necesarios para el uso de un modelo largo de lenguaje.

**Difusers** es otra librería, de hugging face, que es se utiliza como referencia para modelos de difusión previamente entrenados de última generación para generar imágenes, audio e incluso estructuras 3D de moléculas.

**IPython**, como librería, es la arquitectura sobre la cual trabaja un cuaderno de Jupyter, o sea, da la facilidad de interacción que ya se está usa en la herramienta actual... pero se puede usar como libería también para facilitar, en este caso, la muestra de imágenes desde la ejecución de códigos.

Estas herramientas actualmente ya están instaladas en el entorno virtual, y por eso ya no se debe proceder a instalar las liberías para su uso con Python, eso se antes se hacía con el administrador de paquetes pip

```
!pip install transformers
!pip install diffuser
```

# Integrar las herramientas necesarias

Una vez instaladas las herramientas se deben utilizar en los códigos de programación, para ello se utilizan las sentencias de import, con la que se agrega la facultad al programa de poder utilizarlas

In [ ]:
import torch                                   # Activar computación de Tensores con aceleración de GPU
from diffusers import StableDiffusionPipeline  # Atajo para usar el modelo de arte pre-entrenados Stable Difussion
from IPython.display import Image              # Se agrega el componente que permite leer y mostrar imagenes

# Descargar el modelo de Stable Diffusion

Para poder usar el modelo se necesita descargarlo desde la web, este proceso tarda un poco, pero es normal, los modelos para IA en la actualidad son grandes.

In [ ]:
# Se utilizará una variable para que en caso futuro pueda modificarse la ruta o versión del modelo
model_id = "stable-diffusion-v1-5/stable-diffusion-v1-5"
# Se instancia una variable para indicar la descarga y forma de uso del modelo preentrenado
sd_pipeline = StableDiffusionPipeline.from_pretrained( model_id, torch_dtype=torch.float16 )

Luego de la descarga, hay que indicarle al modelo que tipo de arquitectura de procesamiento debe realizar, como la máquina virtual se configuró para que poseea tarjeta gráfica especial, se debe indicarle el tipo de aceleración que debera utilizar el modelo para generar imágenes.

In [ ]:
# Se le indica que formato de procesador de aceleración debe usar para generar las imágenes
sd_pipeline = sd_pipeline.to( "cuda" )

# Facilitando el uso del modelo Stable Diffusion

Es de gran utilidad crear funciones, dado que facilitan el uso de conjuntos de instrucciones grandes donde utilizaremos pocos valores, valores que se pasarán como parámetros a la función, para usar el modelo de Stable Difussion con la tubería especial, lo bueno sería enviarle un prompt, pero también se le dará un prompt negativo para evitar que genere imágenes de mala calidad o muy extrañas

In [ ]:
# Definir una función para indicarle al modelo que nos de una imagen mediante una orden
def obtener_imagen( orden ) :
    # Crear un prompt negativo, una lista de cosas que no queremos que considere para generar una imagen
    lo_que_no_queremos = """
    Bad anatomy,Bad hands,Amputee,Missing fingers,Missing hands,Missing limbs,
    Missing arms,Extra fingers,Extra hands,Extra limbs,Mutated hands,Mutated,
    Mutation,Multiple heads,Malformed limbs,Disfigured,Poorly drawn hands,
    Poorly drawn face,Long neck,Fused fingers,Fused hands,Dismembered,
    Duplicate,Improper scale,Ugly body,Cloned face,Cloned body,Error,
    Gross proportions,Body horror,Too many fingers,simple_background,Pixelated,
    Croped,Out of frame,Out of focus,Dehydrated,lowest quality,Collage,Pixel,
    Bad proportions,extra digits,lowers,username,Disfigured,Disgusting,
    Low res,Low quality,Jpeg,Jpeg artifacts,artist name,error,watermark,
    signature,text,mutated hands,extra digit,fewer digits,worst quality,blury
    """
    # Devolver el resultado de la orden directamente con la imagen generada
    return sd_pipeline( orden, negative_prompt = lo_que_no_queremos ).images[0]

# Probando el funcionamiento

Con la función ya creada, se debe probar, utilizando un prompot simple, se le indicará en una variable que se desea generar y se usará la función para que devuelva la imagen, la guarde en la máquina virtual y luego la muestre

In [ ]:
# Se crea una variable con el prompt
prompt = """
Gato pirata en lancha de madera
"""
# Se genera la imagen utilizando la función que se preparó previamente
imagen_generada = obtener_imagen( prompt )
# Guardar la imagen en la máquina virtual
imagen_generada.save( "./llama.jpg" )
# Mostrar el contenido del prompt que se usó
print( prompt )
# Mostrar la imagen guardada
Image("./llama.jpg")

# Lanzándola a la web

Para permitir que el uso sea mucho más fácil, y además poder compartir con otras personas la creación de imágenes, se puede lograr mediante una aplicación web, y para eso se necesita una herramienta para crear ese tipo de aplicaciones

# Instalando la herramienta para crear aplicaciones web

Se necesitará una herramienta que facilite crear interfaces amigables de tipo web, y gradio es otra librería que se puede utilizar para facilitar esa tarea, se instala con el administrador de paquetes pip

In [ ]:
# Instalar otro paquete para python en la máquina virtual, pero con una versión específica
!pip install gradio==3.48.0

# Integrando las herramientas para la web

Para utilizarla se importará como una libería, pero también se necesita otra librería especial para indicarle que la codificación del texto debe ser aceptable y con símbolos reconocibles en la web (*UTF-8*)

In [ ]:
# Importar
import locale        # Librería de configuración formato de carácteres para la web
import gradio as gr  # Librería que facilita la creación de aplicaciones web

# Eligiendo la codificación de textos de la aplicación web

Se debe indicar que se usará la codificación especial UTF-8 a todos los textos que se generen para ser mostrados y utilizados en las interfaces web.

In [ ]:
# Definir la codificación de caractéres a utilizar con el formato UTF-8
locale.getpreferredencoding = lambda: "UTF-8"

# Diseñando la interfaz de la aplicación web

Para poder crear una interfaz web lo correcto es diseñar primero la idea:
* Se necesitará inicialmente un espacio (textarea) para poder escribir el prompt de entrada
* También deberá existir un espacio (imagen) para que se pueda mostrar la imagen generada
* Asingar un título no es mala idea, identificará a la aplicación web
* Y también una descripción de la aplicación para darle elegancia
* Se puede indicar como serían unos ejemplos de prompt a utiizar

Todos estos elementos serán útiles, pero realmente, lo mas importante es saber a que función se le hará responsable de recibir los valores, y que también nos devolverá una respuesta, la función a utilizar será la que se denominó como **obtener_imagen**

In [ ]:
# Diseño de la aplicación web
# Una variable que definirá una caja para introducir texto
caja_de_entrada  = gr.Textbox( label = "Escribe Tu Prompt Aqui" )
# Otra variable que definirá una caja para mostrar texto
caja_de_salida   = gr.Image(   label = "Resultado"              )
# Una variable más para darle un título a la aplicación web
titulo_de_mi_app = "Generador de Imágenes Cheveres"
# Podría ser útil usar una variable para mostrar la descripción de la aplicación
descripcion_app  = "Herramienta para generar imágenes con Stable Diffusion"
# Un conjunto de ejemplos para poder utilizar como entradas de textos
ejemplos         = [
    "Nave espacial de hormigas exploradoras" ,
    "Barco de conejos cruzando el océano"
]
# La agrupación de todo en una sola interfaz de gradio
interfaz = gr.Interface(
    fn             = obtener_imagen      ,
    inputs         = [ caja_de_entrada ] ,
    outputs        = [ caja_de_salida  ] ,
    title          = titulo_de_mi_app    ,
    description    = descripcion_app     ,
    examples       = ejemplos            ,
    allow_flagging = "never"               # Desaparece un botón llamado Flag
)

# Utilizando la aplicación web

El diseño que ya está listo debe ser ejecutado y puede ser también publicado, la herramienta grado permite utilziar la interfaz creada a travez de un tunel, publicando la interfaz y haciendola accesible a mas usuarios a través de Internet por lo que brindará una url para poder compartirla

In [ ]:
#Lanzamiento de la aplicación web
interfaz.launch( share = True )

# Finalizando la aplicación web

Cuando ya se ha probado la aplicación es correcto detener el servicio para que no se desperdicie el tiempo de uso, tanto de las unidades de CPU de Google Colab, como el servicio de publicación de la herramienta del tunel al sitio de gradio

In [ ]:
# Detener la aplciación web
interfaz.close()